In [1]:
import json
import pandas as pd
import re
import os
import argparse
from typing import Dict, List, Any

# 提取诊断结果的函数
def extract_diagnosis_result(diagnosis_text: str) -> str:
    """
    从诊断文本中提取<box></box>中的诊断结果
    """
    # 查找 <box>xxx</box> 格式的诊断结果
    pattern = r'<box>([^<]+)</box>'
    match = re.search(pattern, diagnosis_text)
    if match:
        return match.group(1).strip()
    return "others"  # 如果没有找到，返回默认值

# ICD代码到新标签的映射函数
def map_icd_to_new_label(icd_code: str) -> str:
    """
    将原来的ICD代码映射到新的诊断标签
    """
    mapping = {
        "F32": "抑郁",
        "F41": "焦虑", 
        "F32,F41": "mix",
        "Others": "others"
    }
    return mapping.get(icd_code, "others")

# 从原始回答中提取ICD代码并映射到新标签
def extract_and_map_ground_truth(gpt_content: str) -> str:
    """
    从GPT回答中提取ICD代码并映射到新的诊断标签
    如果回答中包含新格式的<box>标签，直接提取；否则从ICD代码映射
    """
    # 首先尝试提取新格式的<box>标签
    box_result = extract_diagnosis_result(gpt_content)
    if box_result != "others":  # 如果找到了有效的box标签
        return box_result
    
    # 如果没有找到box标签，尝试从ICD代码映射
    icd_pattern = r'\\icd_code\{([^}]+)\}'
    icd_match = re.search(icd_pattern, gpt_content)
    if icd_match:
        icd_code = icd_match.group(1)
        return map_icd_to_new_label(icd_code)
    
    return "others"

# 数据转换函数
def make_map_fn(data_source: str, split: str):
    """
    创建数据转换函数
    """
    def process_fn(data_item: Dict[str, Any], idx: int) -> Dict[str, Any]:
        patient_id = data_item['patient_id']
        conversations = data_item['conversations']
        
        # 新的system和user prompt
        system_prompt = r"""你是一位经验丰富的精神科医生。请阅读以下病人来精神科问诊的对话记录，并判断病人的主要心理健康状况。抑郁症以**持续的情绪低落、兴趣减退和精力缺乏**为主，而焦虑症则以**过度担忧、紧张不安和对未来事件的恐惧**为主要特点。"""
        
        human_content = ""
        gpt_content = ""
        
        for conv in conversations:
            if conv['from'] == 'human':
                human_content = conv['value']
            elif conv['from'] == 'gpt':
                gpt_content = conv['value']
        
        # 提取对话文本（去掉原有的prompt前缀）
        text = human_content
        # 提取[问诊对话开始]到[问诊对话结束]之间的内容
        start_marker = "[问诊对话开始]"
        end_marker = "[问诊对话结束]"
        start_idx = text.find(start_marker)
        end_idx = text.find(end_marker)
        if start_idx != -1 and end_idx != -1:
            text = text[start_idx + len(start_marker):end_idx].strip()
        
        user_prompt = f"""
[问诊对话开始]
{text}
[问诊对话结束]

请从以下四个选项中选择最合适的诊断：
- 抑郁：主要表现为抑郁症状, 满足ICD诊断要求。 
- 焦虑：主要表现为焦虑症状, 满足ICD诊断要求。 
- mix：同时表现出明显的抑郁和焦虑症状均满足ICD诊断要求，或者都没有满足单诊断为抑郁和焦虑的程度。
- others：其他心理健康问题（比如双向情感障碍，精神分裂症，等等）或正常状态。

请一步一步思考，将思考过程放在<think></think>标签中，最后再将结果（"抑郁"、"焦虑"、"mix"或"others"）放在<box></box>中输出。"""
        
        # 构建prompt - 按照HuggingFace chat template格式
        prompt = [
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user", 
                "content": user_prompt
            }
        ]
        
        # 提取ground truth（映射到新标签）
        ground_truth = extract_and_map_ground_truth(gpt_content)
        
        # 构建转换后的数据
        converted_data = {
            "data_source": data_source,
            "prompt": prompt,
            "ability": "medical_diagnosis",  # 医疗诊断能力
            "reward_model": {
                "style": "rule",
                "ground_truth": ground_truth
            },
            "extra_info": {
                "split": split,
                "index": idx,
                "patient_id": patient_id,
                "original_response": gpt_content
            }
        }
        
        return converted_data
    
    return process_fn

def validate_files(files: List[str]) -> List[str]:
    """
    验证文件列表，移除不存在的文件
    """
    valid_files = []
    for file_path in files:
        if os.path.exists(file_path):
            valid_files.append(file_path)
            print(f"✓ 文件验证通过: {file_path}")
        else:
            print(f"✗ 文件不存在，已跳过: {file_path}")
    return valid_files

def remove_duplicates_by_patient_id(data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    根据patient_id去除重复数据，保留第一次出现的数据
    """
    seen_ids = set()
    deduplicated_data = []
    
    for item in data:
        patient_id = item.get('patient_id', '')
        if patient_id not in seen_ids:
            seen_ids.add(patient_id)
            deduplicated_data.append(item)
        else:
            print(f"发现重复数据，已跳过: {patient_id}")
    
    return deduplicated_data

def convert_sharegpt_to_parquet(train_files: List[str], val_files: List[str], output_dir: str, 
                               remove_duplicates: bool = True):
    """
    将ShareGPT格式的数据转换为后训练格式的parquet文件
    支持多个训练文件和验证文件输入
    
    Args:
        train_files: 训练数据文件路径列表
        val_files: 验证数据文件路径列表 
        output_dir: 输出目录
        remove_duplicates: 是否根据patient_id去除重复数据
    """
    data_source = "SMHC_SFT_auxiliary_diagnosis"
    
    # 验证文件存在性
    print("正在验证文件...")
    train_files = validate_files(train_files)
    val_files = validate_files(val_files)
    
    if not train_files:
        raise ValueError("没有找到有效的训练文件")
    if not val_files:
        raise ValueError("没有找到有效的验证文件")
    
    # 读取多个训练数据文件
    train_data = []
    total_train_items = 0
    for train_file in train_files:
        print(f"正在读取训练数据: {train_file}")
        with open(train_file, 'r', encoding='utf-8') as f:
            file_data = json.load(f)
            train_data.extend(file_data)
            total_train_items += len(file_data)
            print(f"  读取到 {len(file_data)} 条数据")
    
    # 读取多个验证数据文件
    val_data = []
    total_val_items = 0
    for val_file in val_files:
        print(f"正在读取验证数据: {val_file}")
        with open(val_file, 'r', encoding='utf-8') as f:
            file_data = json.load(f)
            val_data.extend(file_data)
            total_val_items += len(file_data)
            print(f"  读取到 {len(file_data)} 条数据")
    
    print(f"\n合并后数据统计:")
    print(f"训练数据: {total_train_items} 条 (来自 {len(train_files)} 个文件)")
    print(f"验证数据: {total_val_items} 条 (来自 {len(val_files)} 个文件)")
    
    # 去重处理
    if remove_duplicates:
        print("\n正在进行数据去重...")
        original_train_count = len(train_data)
        original_val_count = len(val_data)
        
        train_data = remove_duplicates_by_patient_id(train_data)
        val_data = remove_duplicates_by_patient_id(val_data)
        
        print(f"训练数据去重: {original_train_count} -> {len(train_data)} (移除 {original_train_count - len(train_data)} 条)")
        print(f"验证数据去重: {original_val_count} -> {len(val_data)} (移除 {original_val_count - len(val_data)} 条)")
    
    # 创建转换函数
    train_map_fn = make_map_fn(data_source, 'train')
    val_map_fn = make_map_fn(data_source, 'val')
    
    # 转换数据
    print("正在转换训练数据...")
    converted_train = [train_map_fn(item, idx) for idx, item in enumerate(train_data)]
    
    print("正在转换验证数据...")
    converted_val = [val_map_fn(item, idx) for idx, item in enumerate(val_data)]
    
    # 创建输出目录
    os.makedirs(output_dir, exist_ok=True)
    
    # 转换为DataFrame并保存为parquet
    print("正在保存为parquet格式...")
    train_df = pd.DataFrame(converted_train)
    val_df = pd.DataFrame(converted_val)
    
    train_output_path = os.path.join(output_dir, 'train.parquet')
    val_output_path = os.path.join(output_dir, 'val.parquet')
    
    train_df.to_parquet(train_output_path, index=False)
    val_df.to_parquet(val_output_path, index=False)
    
    print(f"训练数据已保存至: {train_output_path}")
    print(f"验证数据已保存至: {val_output_path}")
    print(f"训练数据量: {len(converted_train)} 条")
    print(f"验证数据量: {len(converted_val)} 条")
    
    # 显示样例数据
    print("\n样例数据 (训练集第一条):")
    print(str(json.dumps(converted_train[0], ensure_ascii=False, indent=2)))
    
    return train_df, val_df

def add_multiple_files_example():
    """
    示例：如何添加多个文件
    """
    # 方式1：直接列出所有文件
    train_files = [
        "/tcci_mnt/shihao/project/LLaMA-Factory/shihao/data/SMHC_SFT_auxiliary_diagnosis_training_data_v4_train.json",
        # "/path/to/additional_train_file1.json",
        # "/path/to/additional_train_file2.json",
    ]
    
    val_files = [
        "/tcci_mnt/shihao/project/LLaMA-Factory/shihao/data/SMHC_SFT_auxiliary_diagnosis_training_data_v4_val.json",
        # "/path/to/additional_val_file1.json", 
        # "/path/to/additional_val_file2.json",
    ]
    
    return train_files, val_files

def get_files_from_pattern(pattern: str) -> List[str]:
    """
    根据文件模式获取文件列表
    """
    import glob
    files = glob.glob(pattern)
    files.sort()  # 确保文件顺序一致
    return files

# 设置文件路径 - 支持多文件输入
train_files, val_files = add_multiple_files_example()

# 你也可以使用文件模式匹配（如果有多个相似命名的文件）
# train_files = get_files_from_pattern("/path/to/train_data_*.json")
# val_files = get_files_from_pattern("/path/to/val_data_*.json")

output_dir = "/tcci_mnt/shihao/project/verl/psy_r1/SMHC_data_v4"

# 配置选项
REMOVE_DUPLICATES = True  # 是否去除重复数据

# 执行转换
print("开始执行完整数据转换...")
print(f"训练文件数量: {len(train_files)}")
print(f"验证文件数量: {len(val_files)}")
print(f"去重选项: {'启用' if REMOVE_DUPLICATES else '禁用'}")
print("-" * 50)

train_df, val_df = convert_sharegpt_to_parquet(
    train_files=train_files, 
    val_files=val_files, 
    output_dir=output_dir,
    remove_duplicates=REMOVE_DUPLICATES
)


开始执行完整数据转换...
训练文件数量: 1
验证文件数量: 1
去重选项: 启用
--------------------------------------------------
正在验证文件...
✓ 文件验证通过: /tcci_mnt/shihao/project/LLaMA-Factory/shihao/data/SMHC_SFT_auxiliary_diagnosis_training_data_v4_train.json
✓ 文件验证通过: /tcci_mnt/shihao/project/LLaMA-Factory/shihao/data/SMHC_SFT_auxiliary_diagnosis_training_data_v4_val.json
正在读取训练数据: /tcci_mnt/shihao/project/LLaMA-Factory/shihao/data/SMHC_SFT_auxiliary_diagnosis_training_data_v4_train.json
  读取到 13428 条数据
正在读取验证数据: /tcci_mnt/shihao/project/LLaMA-Factory/shihao/data/SMHC_SFT_auxiliary_diagnosis_training_data_v4_val.json
  读取到 1492 条数据

合并后数据统计:
训练数据: 13428 条 (来自 1 个文件)
验证数据: 1492 条 (来自 1 个文件)

正在进行数据去重...
训练数据去重: 13428 -> 13428 (移除 0 条)
验证数据去重: 1492 -> 1492 (移除 0 条)
正在转换训练数据...
正在转换验证数据...
正在保存为parquet格式...
训练数据已保存至: /tcci_mnt/shihao/project/verl/psy_r1/SMHC_data_v4/train.parquet
验证数据已保存至: /tcci_mnt/shihao/project/verl/psy_r1/SMHC_data_v4/val.parquet
训练数据量: 13428 条
验证数据量: 1492 条

样例数据 (训练集第一条):
{
  "data_source": "SMHC_SFT_a

In [2]:
# 验证和使用生成的parquet文件

# 1. 读取parquet文件
import pandas as pd
import json

# 读取生成的parquet文件
train_df = pd.read_parquet('/tcci_mnt/shihao/project/verl/psy_r1/SMHC_data_v4/train.parquet')
val_df = pd.read_parquet('/tcci_mnt/shihao/project/verl/psy_r1/SMHC_data_v4/val.parquet')

print(f"训练集数据量: {len(train_df)}")
print(f"验证集数据量: {len(val_df)}")
print("\n数据列名:")
print(train_df.columns.tolist())

# 2. 检查数据结构
print("\n数据样例:")
sample = train_df.iloc[0]

# 安全地显示样例数据，避免numpy数组序列化问题
print("第一条数据的结构:")
print(f"- data_source: {sample['data_source']}")
print(f"- ability: {sample['ability']}")
print(f"- prompt: 包含 {len(sample['prompt'])} 个消息")
print(f"- reward_model: {sample['reward_model']}")
print(f"- extra_info: {sample['extra_info']}")

# 显示prompt的内容（截取前200个字符）
print(f"\n- prompt内容预览:")
prompt_content = sample['prompt'][0]['content'][:200] + "..." if len(sample['prompt'][0]['content']) > 200 else sample['prompt'][0]['content']
print(f"  角色: {sample['prompt'][0]['role']}")
print(f"  内容: {prompt_content}")

# 3. 统计不同诊断类别的分布
print("\n诊断类别分布:")

# 训练集诊断分布
train_diagnosis = train_df['reward_model'].apply(lambda x: x['ground_truth'])
print("训练集:")
print(train_diagnosis.value_counts())

# 验证集诊断分布
val_diagnosis = val_df['reward_model'].apply(lambda x: x['ground_truth'])
print("\n验证集:")
print(val_diagnosis.value_counts())

# 4. 检查数据质量
print("\n数据质量检查:")
print(f"训练集空值检查: {train_df.isnull().sum().sum()}")
print(f"验证集空值检查: {val_df.isnull().sum().sum()}")

# 5. 示例：如何使用数据进行后训练
print("\n后训练数据使用示例:")
print("每条数据包含以下字段:")
print("- data_source: 数据来源")
print("- prompt: HuggingFace chat template格式的对话")
print("- ability: 任务类型 (medical_diagnosis)")
print("- reward_model: 包含ground_truth的奖励模型信息")
print("- extra_info: 额外信息 (split, index, patient_id, original_response)")

print("\n转换完成！数据已准备好用于后训练。")


训练集数据量: 13428
验证集数据量: 1492

数据列名:
['data_source', 'prompt', 'ability', 'reward_model', 'extra_info']

数据样例:
第一条数据的结构:
- data_source: SMHC_SFT_auxiliary_diagnosis
- ability: medical_diagnosis
- prompt: 包含 2 个消息
- reward_model: {'ground_truth': '抑郁', 'style': 'rule'}
- extra_info: {'index': 0, 'original_response': '<box>抑郁</box>', 'patient_id': '337_conv5', 'split': 'train'}

- prompt内容预览:
  角色: system
  内容: 你是一位经验丰富的精神科医生。请阅读以下病人来精神科问诊的对话记录，并判断病人的主要心理健康状况。抑郁症以**持续的情绪低落、兴趣减退和精力缺乏**为主，而焦虑症则以**过度担忧、紧张不安和对未来事件的恐惧**为主要特点。

诊断类别分布:
训练集:
reward_model
抑郁        5787
others    3465
mix       3096
焦虑        1080
Name: count, dtype: int64

验证集:
reward_model
抑郁        643
others    385
mix       344
焦虑        120
Name: count, dtype: int64

数据质量检查:
训练集空值检查: 0
验证集空值检查: 0

后训练数据使用示例:
每条数据包含以下字段:
- data_source: 数据来源
- prompt: HuggingFace chat template格式的对话
- ability: 任务类型 (medical_diagnosis)
- reward_model: 包含ground_truth的奖励模型信息
- extra_info: 额外信息 (split, index, patient_id, original_response)

转换完成！数据已

In [3]:
# 多文件输入的使用示例

print("=== 多文件输入功能演示 ===\n")

# 示例1：手动指定多个文件
def example_manual_files():
    """手动指定多个JSON文件"""
    train_files = [
        "/path/to/dataset1_train.json",
        "/path/to/dataset2_train.json", 
        "/path/to/dataset3_train.json",
    ]
    
    val_files = [
        "/path/to/dataset1_val.json",
        "/path/to/dataset2_val.json",
        "/path/to/dataset3_val.json", 
    ]
    
    return train_files, val_files

# 示例2：使用文件模式匹配
def example_pattern_matching():
    """使用glob模式匹配文件"""
    import glob
    
    # 匹配所有以特定模式命名的训练文件
    train_files = get_files_from_pattern("/path/to/*_train.json")
    val_files = get_files_from_pattern("/path/to/*_val.json")
    
    # 或者匹配特定目录下的所有JSON文件
    # train_files = get_files_from_pattern("/path/to/train_data/*.json")
    # val_files = get_files_from_pattern("/path/to/val_data/*.json")
    
    return train_files, val_files

# 示例3：从目录中读取所有JSON文件
def example_directory_scan(train_dir: str, val_dir: str):
    """扫描目录中的所有JSON文件"""
    import os
    
    train_files = []
    val_files = []
    
    # 扫描训练数据目录
    if os.path.exists(train_dir):
        for filename in sorted(os.listdir(train_dir)):
            if filename.endswith('.json'):
                train_files.append(os.path.join(train_dir, filename))
    
    # 扫描验证数据目录  
    if os.path.exists(val_dir):
        for filename in sorted(os.listdir(val_dir)):
            if filename.endswith('.json'):
                val_files.append(os.path.join(val_dir, filename))
                
    return train_files, val_files

# 示例4：交互式文件选择
def example_interactive_selection():
    """交互式添加文件"""
    train_files = []
    val_files = []
    
    print("请输入训练文件路径（输入'done'结束）:")
    while True:
        file_path = input("训练文件: ").strip()
        if file_path.lower() == 'done':
            break
        if file_path and os.path.exists(file_path):
            train_files.append(file_path)
            print(f"✓ 已添加: {file_path}")
        elif file_path:
            print(f"✗ 文件不存在: {file_path}")
    
    print("\n请输入验证文件路径（输入'done'结束）:")
    while True:
        file_path = input("验证文件: ").strip()
        if file_path.lower() == 'done':
            break
        if file_path and os.path.exists(file_path):
            val_files.append(file_path)
            print(f"✓ 已添加: {file_path}")
        elif file_path:
            print(f"✗ 文件不存在: {file_path}")
            
    return train_files, val_files

print("使用方式：")
print("1. 手动指定文件列表:")
print("   train_files, val_files = example_manual_files()")
print()
print("2. 使用模式匹配:")
print("   train_files, val_files = example_pattern_matching()")
print()
print("3. 扫描目录:")
print("   train_files, val_files = example_directory_scan('/train/dir', '/val/dir')")
print()
print("4. 交互式选择:")
print("   train_files, val_files = example_interactive_selection()")
print()
print("然后调用:")
print("   convert_sharegpt_to_parquet(train_files, val_files, output_dir)")

# 检查当前使用的文件
print(f"\n当前配置的文件:")
print(f"训练文件 ({len(train_files)}):")
for i, f in enumerate(train_files, 1):
    print(f"  {i}. {f}")
    
print(f"\n验证文件 ({len(val_files)}):")
for i, f in enumerate(val_files, 1):
    print(f"  {i}. {f}")

print(f"\n总计: {len(train_files)} 个训练文件, {len(val_files)} 个验证文件")


=== 多文件输入功能演示 ===

使用方式：
1. 手动指定文件列表:
   train_files, val_files = example_manual_files()

2. 使用模式匹配:
   train_files, val_files = example_pattern_matching()

3. 扫描目录:
   train_files, val_files = example_directory_scan('/train/dir', '/val/dir')

4. 交互式选择:
   train_files, val_files = example_interactive_selection()

然后调用:
   convert_sharegpt_to_parquet(train_files, val_files, output_dir)

当前配置的文件:
训练文件 (1):
  1. /tcci_mnt/shihao/project/LLaMA-Factory/shihao/data/SMHC_SFT_auxiliary_diagnosis_training_data_v4_train.json

验证文件 (1):
  1. /tcci_mnt/shihao/project/LLaMA-Factory/shihao/data/SMHC_SFT_auxiliary_diagnosis_training_data_v4_val.json

总计: 1 个训练文件, 1 个验证文件


In [4]:
# 最佳实践示例：多文件数据处理完整流程

print("=== 多文件数据处理最佳实践 ===\n")

def advanced_multi_file_processing():
    """
    高级多文件处理示例，包含完整的数据处理流程
    """
    
    # 1. 定义数据源配置
    data_sources = {
        "primary": {
            "train": ["/tcci_mnt/shihao/project/LLaMA-Factory/shihao/data/SMHC_SFT_auxiliary_diagnosis_training_data_v4_train.json"],
            "val": ["/tcci_mnt/shihao/project/LLaMA-Factory/shihao/data/SMHC_SFT_auxiliary_diagnosis_training_data_v4_val.json"],
        },
        # 可以添加更多数据源
        # "secondary": {
        #     "train": ["/path/to/secondary_train1.json", "/path/to/secondary_train2.json"],
        #     "val": ["/path/to/secondary_val1.json"],
        # },
    }
    
    # 2. 合并所有数据源
    all_train_files = []
    all_val_files = []
    
    for source_name, files in data_sources.items():
        print(f"数据源: {source_name}")
        print(f"  训练文件: {len(files['train'])} 个")
        print(f"  验证文件: {len(files['val'])} 个")
        
        all_train_files.extend(files['train'])
        all_val_files.extend(files['val'])
    
    print(f"\n总计:")
    print(f"  训练文件: {len(all_train_files)} 个")
    print(f"  验证文件: {len(all_val_files)} 个")
    
    return all_train_files, all_val_files

def batch_process_multiple_datasets():
    """
    批量处理多个数据集的示例
    """
    datasets = [
        {
            "name": "SMHC_v4",
            "train_files": ["/tcci_mnt/shihao/project/LLaMA-Factory/shihao/data/SMHC_SFT_auxiliary_diagnosis_training_data_v4_train.json"],
            "val_files": ["/tcci_mnt/shihao/project/LLaMA-Factory/shihao/data/SMHC_SFT_auxiliary_diagnosis_training_data_v4_val.json"],
            "output_dir": "/tcci_mnt/shihao/project/verl/psy_r1/SMHC_data_v4"
        },
        # 可以添加更多数据集
        # {
        #     "name": "additional_dataset",
        #     "train_files": ["/path/to/additional_train.json"],
        #     "val_files": ["/path/to/additional_val.json"],
        #     "output_dir": "/path/to/additional_output"
        # }
    ]
    
    for dataset in datasets:
        print(f"\n处理数据集: {dataset['name']}")
        print("-" * 40)
        
        try:
            train_df, val_df = convert_sharegpt_to_parquet(
                train_files=dataset['train_files'],
                val_files=dataset['val_files'], 
                output_dir=dataset['output_dir'],
                remove_duplicates=True
            )
            print(f"✓ {dataset['name']} 处理完成")
            
        except Exception as e:
            print(f"✗ {dataset['name']} 处理失败: {e}")

# 数据质量检查函数
def check_data_quality(files: List[str]) -> Dict[str, Any]:
    """
    检查数据质量
    """
    quality_report = {
        "total_files": len(files),
        "valid_files": 0,
        "total_records": 0,
        "patient_ids": set(),
        "file_details": []
    }
    
    for file_path in files:
        file_info = {"path": file_path, "exists": False, "records": 0, "error": None}
        
        try:
            if os.path.exists(file_path):
                file_info["exists"] = True
                quality_report["valid_files"] += 1
                
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    file_info["records"] = len(data)
                    quality_report["total_records"] += len(data)
                    
                    # 收集patient_id
                    for item in data:
                        if 'patient_id' in item:
                            quality_report["patient_ids"].add(item['patient_id'])
            else:
                file_info["error"] = "文件不存在"
                
        except Exception as e:
            file_info["error"] = str(e)
            
        quality_report["file_details"].append(file_info)
    
    quality_report["unique_patients"] = len(quality_report["patient_ids"])
    return quality_report

# 演示用法
print("使用高级多文件处理:")
train_files, val_files = advanced_multi_file_processing()

print("\n数据质量检查:")
train_quality = check_data_quality(train_files)
val_quality = check_data_quality(val_files)

print(f"训练数据质量报告:")
print(f"  有效文件: {train_quality['valid_files']}/{train_quality['total_files']}")
print(f"  总记录数: {train_quality['total_records']}")
print(f"  唯一患者: {train_quality['unique_patients']}")

print(f"验证数据质量报告:")
print(f"  有效文件: {val_quality['valid_files']}/{val_quality['total_files']}")
print(f"  总记录数: {val_quality['total_records']}")
print(f"  唯一患者: {val_quality['unique_patients']}")

print("\n要执行批量处理，取消注释下面的代码:")
print("# batch_process_multiple_datasets()")


=== 多文件数据处理最佳实践 ===

使用高级多文件处理:
数据源: primary
  训练文件: 1 个
  验证文件: 1 个

总计:
  训练文件: 1 个
  验证文件: 1 个

数据质量检查:
训练数据质量报告:
  有效文件: 1/1
  总记录数: 13428
  唯一患者: 13428
验证数据质量报告:
  有效文件: 1/1
  总记录数: 1492
  唯一患者: 1492

要执行批量处理，取消注释下面的代码:
# batch_process_multiple_datasets()
